## Setup Notebook (Required to run this from the local directory where the dataset has been downloaded)

1. This notebook is a *PRE-REQUISITE* for the 1_Distributed_Model_Training_Snowflake_Notebooks.ipynb notebook. 
2. The solution leverages an open source dataset that is available [here](https://github.com/Charmve/Surface-Defect-Detection/tree/master/DeepPCB/PCBData) for downloading. Be sure to review and comply with the licensing terms and usage guidelines of that repository owner before utilizing the dataset. 

#### After cloning the DeepPCB dataset to your local, use this notebook to extract the images, convert into base64 code and load the contents along with the label information into Snowflake Stage
 

#### Import necessary libraries. Remember to refer to the env.yaml for the complete list of libraries.

In [1]:
from snowflake.snowpark.session import Session

import getpass
import pandas as pd
import json

In [2]:
DATABASE = 'PCB_CV'
SCHEMA = 'PUBLIC'
ROLE = 'PCB_CV_ROLE'
WAREHOUSE = 'PCB_CV_WH'
COMPUTEPOOL = 'PCB_CV_COMPUTEPOOL'
STAGE = 'PCB_CV_DEEP_PCB_DATASET_STAGE'

## Establish connection to Snowflake

In [3]:
session = Session.builder.config("connection_name", "demo").create()
session.use_database(DATABASE)
session.use_schema(SCHEMA)
session.use_role(ROLE)

In [9]:
d = session.sql("select current_user()")
d.collect()

[Row(CURRENT_USER()='TRSMITH')]

#### Process the folder structure in the original repository to identify the appropriate .jpg and .txt files in the PCBData folder and upload them to the specified Snowflake stages. The script uses the os module to traverse the directory and the Snowflake Python connector to handle file uploads.

In [ ]:
import os

def upload_to_snowflake(stage, file_path, file_type):
    """
    Upload a file to the specified Snowflake stage.

    Args:
        stage (str): Snowflake stage path (e.g., @data_stage/images/train).
        file_path (str): Path to the file to upload.
        file_type (str): Type of file being uploaded 
    """
    try:
        
        session.file.put(f"file://{file_path}", stage,auto_compress= False)
        #session.sql(f"PUT 'file://{file_path}' {stage}")
        print(f"Uploaded file: {file_path} to {stage}")
        return 0
    except Exception as e:
        print(f"Failed to upload  file: {file_path}. Error: {str(e)}")
        return 1

def process_and_upload_files(base_dir, image_stage, label_stage):
    """
    Process the directory structure and upload _test.jpg files and their corresponding .txt files to Snowflake.

    Args:
        base_dir (str): Path to the base directory containing PCB data.
        image_stage (str): Snowflake stage path for images.
        label_stage (str): Snowflake stage path for labels.
    """
    for group_folder in os.listdir(base_dir):
        group_path = os.path.join(base_dir, group_folder)
        if not os.path.isdir(group_path):
            continue

        for sub_folder in os.listdir(group_path):
            sub_folder_path = os.path.join(group_path, sub_folder)
            

            if not os.path.isdir(sub_folder_path):
                continue

            if sub_folder.endswith("_not"):
                continue

            folder_not = os.path.join(group_path, sub_folder + "_not")

            if not os.path.exists(folder_not):
                continue

            for file_name in os.listdir(sub_folder_path):
                if file_name.endswith("_test.jpg"):
                    
                    # Full path of the .jpg file
                    jpg_file_path = os.path.join(sub_folder_path, file_name)
                    
                    # Corresponding .txt file path
                    txt_file_name = os.path.splitext(file_name.replace("_test", ""))[0] + ".txt"
                    txt_file_path = os.path.join(folder_not, txt_file_name)
                    print(txt_file_path)
                    if os.path.exists(txt_file_path):
                        # Upload the .jpg file to the images stage
                        rc = upload_to_snowflake(image_stage_path,jpg_file_path, "image")
                        if rc == 1:
                            printf("Upload Failed.")
                            break

                        # Upload the .txt file to the labels stage
                        rc = upload_to_snowflake(label_stage_path,txt_file_path, "label")
                        if rc == 1:
                            printf("Upload Failed.")
                            break



# Define local directory where the PCB dataset was downloaded
base_directory = "../data/PCBData" #ensure this notebook is in the same directory as the PCBData folder
image_stage_path = f"@{STAGE}/images/train"
label_stage_path = f"@{STAGE}/labels/train"

# Process and upload files
process_and_upload_files(base_directory, image_stage_path, label_stage_path)



../data/PCBData/group44000/44000_not/44000022.txt
Uploaded file: ../data/PCBData/group44000/44000/44000022_test.jpg to @PCB_CV_DEEP_PCB_DATASET_STAGE/images/train
Uploaded file: ../data/PCBData/group44000/44000_not/44000022.txt to @PCB_CV_DEEP_PCB_DATASET_STAGE/labels/train
../data/PCBData/group44000/44000_not/44000023.txt


### Download Labels 
Download label files from the Snowflake stage, parse each label text file to extract label data, and then save this data into a DataFrame which is written to a Snowflake table called LABELS_TRAIN.

After successful data upload to the Snowflake Internal Stage, follow the below steps. The label contains the Xmin,Ymin,Xmax and Ymax coorrdinates of the defects and the class of defect that will be used to train this supervised model.

In [27]:
import os
import base64
import pandas as pd
stage_train_path = "@DATA_STAGE/labels/train"
local_train_dir = "/tmp/labels/"


# Ensure local directories exist
os.makedirs(local_train_dir, exist_ok=True)

#train_files = session.file.get(stage_train_path, "/tmp/labels/")
# Download .txt files from the train stage
train_files = session.file.get(stage_train_path, local_train_dir, pattern=".*\.txt")

def download_files_from_stage(stage_path, local_dir):
    
    files = session.file.get(stage_path, local_dir)
    
    # Check if files were downloaded successfully
    if files:
        for file in files:
            
            local_file_path = os.path.join(local_dir, os.path.basename(file.file))
            
    else:
        print(f"No .txt files were downloaded from {stage_path}.")

# Download .txt files from the train stage
download_files_from_stage(stage_train_path, local_train_dir)
path_annot="/tmp/labels/"

# Initialize a list to hold all the data  
data = []  
  
# Walking through the directory to get all label files  
for path, subdirs, files in os.walk(path_annot):  
    for name in files:  
        if name.endswith('.txt'):  # Filter to include only .txt files  
            full_path = os.path.join(path, name)  
            with open(full_path, 'r') as file:  
                for line in file:  
                    parts = line.strip().split()  
                    if len(parts) == 5:
                        xmin, ymin, xmax, ymax,class_id = parts  
                        data.append({  
                            "filename": name.replace('.txt', ''), 
                            "xmin": float(xmin),  
                            "ymin": float(ymin),  
                            "xmax": float(xmax),  
                            "ymax": float(ymax) ,
                            "class": int(class_id)
                        })  
  
# Create a DataFrame  
trainlabels_df = pd.DataFrame(data)
train_labels = session.create_dataframe(trainlabels_df)

train_labels.write.save_as_table("LABELS_TRAIN", mode="overwrite")

### Download Images 
 Downloads image files from a Snowflake stage, encode them in Base64, merges them with label data that was processed in the last step, and inserts the combined information into a Snowflake table named train_images_labels.

In [ ]:

import os
import base64
import pandas as pd

train_images_stage_path = "@DATA_STAGE/images/"


local_train_images_dir = "/tmp/images/train"


os.makedirs(local_train_images_dir, exist_ok=True)

# Function to download images from Snowflake stage and convert them to Base64
def download_images_from_stage(stage_path, local_dir):
    # Use session.file.get() to fetch images
    files = session.file.get(stage_path, local_dir)
    
    # List to hold the image data
    images_data = []
    
    if files:
        for file in files:
            filename = os.path.basename(file.file)            
            # Read the image and convert to Base64
            with open(os.path.join(local_dir, filename), "rb") as image_file:
                encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
                images_data.append({"Filename": filename.replace(".jpg", ""), "image_data": encoded_string})
    else:
        print(f"No image files were downloaded from {stage_path}.")
    
    return images_data

# Download the train and validation images and convert to Base64
train_images_base64_data = download_images_from_stage(train_images_stage_path, local_train_images_dir)

# Convert to DataFrames for inserting into separate tables
train_images_df = pd.DataFrame(train_images_base64_data)

print("trainlabels_df columns:", trainlabels_df.columns)
print("train_images_df columns:", train_images_df.columns)

if 'filename' in trainlabels_df.columns:
    trainlabels_df.rename(columns={'filename': 'Filename'}, inplace=True)
# Update labels DataFrame to include `_test.jpg` suffix for matching
trainlabels_df['Filename'] = trainlabels_df['Filename'] + "_test"

merged_train_df = pd.merge(trainlabels_df, train_images_df, how='inner', on='Filename')



# Create the Snowflake table schema to accommodate both image data and label information
create_train_table_query = """
CREATE OR REPLACE TABLE train_images_labels (
    Filename varchar,
    image_data VARCHAR,
    class INT,
    xmin FLOAT,
    ymin FLOAT,
    xmax FLOAT,
    ymax FLOAT
)
"""
# Execute the table creation query
session.sql(create_train_table_query).collect()

# Function to insert merged image and label data into the specified table
def insert_images_and_labels_into_table(merged_df, table_name):
    for index, row in merged_df.iterrows():
        filename = row['Filename'].replace('_test', '')
        insert_query = f"""
        INSERT INTO {table_name} (Filename, image_data, class, xmin, ymin, xmax, ymax)
        VALUES ('{filename}', '{row['image_data']}', '{row['class']}', {row['xmin']}, {row['ymin']}, {row['xmax']}, {row['ymax']})
        """
        print("")
        session.sql(insert_query).collect()

# Insert merged data into the train table
insert_images_and_labels_into_table(merged_train_df, "train_images_labels")

print("Merged image and label data inserted into table.")


trainlabels_df columns: Index(['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'class'], dtype='object')
train_images_df columns: Index(['Filename', 'image_data'], dtype='object')






In [ ]:
from sklearn.model_selection import train_test_split

df=session.table("TRAIN_IMAGES_LABELS").to_pandas()

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

session.write_pandas(train_df,"training_data", auto_create_table=True, overwrite=True, quote_identifiers=False)

session.write_pandas(test_df,"test_data", auto_create_table=True, overwrite=True,quote_identifiers=False)


## End of Setup and Data Preparation